# T5 Modelini XSum Veri Seti ile Metin Özeti (Text Summarization) İçin Fine-Tune Etme

Bu projede, **t5-small** modeli **XSum** veri seti kullanılarak **metin özeti (text summarization)** görevleri için **fine-tuning** işlemine tabi tutulmuştur.

Model, çeşitli talimatlarla sağlanan metinleri özetleyebilmek için **Seq2SeqTrainer** ve **Seq2SeqTrainingArguments** kullanılarak eğitilmiştir. Eğitimde kullanılan özetleme talimatları, modelin farklı soru biçimleri ve içeriklere adapte olabilmesini sağlamak amacıyla çeşitlendirilmiştir. Bu süreç sonunda model, belgeleri hızlı ve etkili bir şekilde özetleyebilme yeteneği kazanmıştır.

### Veri Seti
**XSum** veri seti, haber makalelerini ve bunların özetlerini içerir. Eğitim verisi, %80 eğitim ve %20 değerlendirme (test) olacak şekilde ayrılmıştır.

### Kullanılan Donanım
- Kaggle Notebook - GPU: Tesla P100


Bu çalışma, metin özeti üretimi gibi doğal dil işleme (NLP) görevleri için modelin etkinliğini artırmayı amaçlamaktadır.

![image](../imgs/summary.ppm)

In [1]:
# GPU ve CUDA sürümlerini kontrol etmek için kullanılır
!nvidia-smi

Sat Oct  5 14:02:43 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P8              9W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# warnings kütüphanesi uyarıları kapatmak için kullanılır
import warnings 
warnings.filterwarnings("ignore")

In [3]:
!pip install -q -U transformers datasets huggingface_hub
!echo "Installations completed!"

Installations completed!


In [4]:
from datasets import load_dataset
import random 

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
import torch

from transformers import pipeline

from huggingface_hub import notebook_login

In [5]:
notebook_login()

## 🤗 Hugging Face Datasets Kütüphanesi

### Parametreler

- **`path`**:
  - Yüklenecek veri setinin yolu veya ismi. Örnek: "imdb", "glue".

- **`name`**:
  - Yüklenecek veri setinin alt kümesi. Örnek: "sst2" (GLUE için).

- **`data_dir`**:
  - Veri dosyalarının bulunduğu dizin.

- **`data_files`**:
  - Yüklenecek veri dosyaları.

- **`split`**:
  - Veri setinin bölünmesi (örneğin "train", "test").

- **`cache_dir`**:
  - Verinin önbelleğe alınacağı dizin.

- **`features`**:
  - Özelliklerin açıkça belirtildiği yer.

- **`download_config`**:
  - İndirme yapılandırma ayarları.

- **`download_mode`**:
  - İndirme modu: "reuse_dataset_if_exists", "reuse_cache_if_exists", "force_redownload".

- **`verification_mode`**:
  - Veri setinin doğrulama modu.

- **`ignore_verifications`**:
  - Artık kullanılmıyor, doğrulamaları atlamak için.

- **`keep_in_memory`**:
  - Eğer True ise, veri seti bellekte tutulur.

- **`save_infos`**:
  - Eğer True ise, veri seti bilgileri kaydedilir.

- **`revision`**:
  - Yüklenecek veri setinin versiyonu veya commit ID'si.

- **`token`**:
  - Private veri setleri için kullanılır, bir token sağlar.

- **`use_auth_token`**:
  - Artık kullanılmıyor, oturum açma token'ı için.

- **`task`**:
  - Artık kullanılmıyor, veri seti yükleme görevini belirtmek için.

- **`streaming`**:
  - Eğer True ise, veri seti akış modunda yüklenir.

- **`num_proc`**:
  - Çok işlemcili veri işleme için işlemci sayısı.

- **`storage_options`**:
  - Depolama seçenekleri.

- **`trust_remote_code`**:
  - Eğer True ise, uzaktan kod çalıştırmaya izin verir.

- **`**config_kwargs`**:
  - Diğer ek yapılandırma argümanları.


In [6]:
dataset = load_dataset('xsum', split='train[:1%]', trust_remote_code=True)

dataset = dataset.train_test_split(train_size=0.8)
train_dataset, eval_dataset = dataset['train'], dataset['test']

xsum.py:   0%|          | 0.00/5.76k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.24k [00:00<?, ?B/s]

(…)SUM-EMNLP18-Summary-Data-Original.tar.gz:   0%|          | 0.00/255M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

In [7]:
print(train_dataset)
print('-')
train_dataset[0]

Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 1632
})
-


{'document': "Corey Price, Alesha O'Connor and Rhodri Miller, all 17, and Margaret Challis, 66, died in a two-car collision on the A470 in the Brecon Beacons in March.\nCardiff City Football Club is hosting the match on Sunday, with teams made up of members of the community.\nCorey was part of the club's academy for eight years from the age of nine.\nEntry to the match at Cardiff City Stadium is free but there will be bucket collections on the day.\nThe money will be used to pay for a Corey Price memorial bench at Ysgol Gyfun Bro Morgannwg, in Barry, Vale of Glamorgan, where he was a pupil.\nA second bench is also planned at the Cardiff City Academy training ground and the club hopes the match will become an annual event.\nCorey and his two friends, all from Barry, were travelling together when the collision happened near Storey Arms, Brecon on 6 March.\nMrs Challis, 66, from Merthyr Tydfil, was a passenger in the second car involved.\nSeven teenage drivers were arrested on suspicion o

## 🤗 Hugging Face Tokenizer Kütüphanesi

### Parametreler

- **`text`**:
  - Tokenize edilecek metin veya metinlerin listesi.

- **`text_pair`**:
  - İkinci bir metin veya metinlerin listesi, çift metinli modeller için kullanılır.

- **`text_target`**:
  - Hedef metin veya metinlerin listesi (genellikle seq2seq modelleri için kullanılır).

- **`text_pair_target`**:
  - Hedef ikinci metin veya metinlerin listesi, çift metinli modeller için kullanılır.

- **`add_special_tokens`**:
  - Özel tokenler ekler (örneğin [CLS], [SEP]). True olması, modeli daha iyi performans gösterir.

- **`padding`**:
  - Padding stratejisi: True, False, "longest", "max_length".

- **`truncation`**:
  - Kesme stratejisi: True, False, "longest_first", "only_first".

- **`max_length`**:
  - Maksimum token sayısı. Yüksek değerler daha fazla bilgi taşır ama daha fazla bellek kullanır.

- **`stride`**:
  - Kesme sırasında kayma boyutu. Uzun metinler için daha küçük değerler kullanışlı olabilir.

- **`is_split_into_words`**:
  - Eğer True ise, metin kelimelere bölünmüş olarak kabul edilir.

- **`pad_to_multiple_of`**:
  - Padding boyutunun bir katı olacak şekilde padding ekler.

- **`return_tensors`**:
  - Döndürülecek tensör tipi: 'pt', 'tf', 'np'.

- **`return_token_type_ids`**:
  - Eğer True ise, token tipi ID'lerini döndürür.

- **`return_attention_mask`**:
  - Eğer True ise, attention maskelerini döndürür.

- **`return_overflowing_tokens`**:
  - Eğer True ise, taşan tokenleri döndürür.

- **`return_special_tokens_mask`**:
  - Eğer True ise, özel token maskesini döndürür.

- **`return_offsets_mapping`**:
  - Eğer True ise, offset mapping döndürür.

- **`return_length`**:
  - Eğer True ise, token uzunluklarını döndürür.

- **`verbose`**:
  - Eğer True ise, işlem hakkında daha fazla bilgi verir.

- **`**kwargs`**:
  - Diğer ek argümanlar.


In [8]:
base_model_name = 't5-small' # Daha fazla model için: https://huggingface.co/models?pipeline_tag=summarization
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = AutoTokenizer.from_pretrained(base_model_name)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

## 🤗 Hugging Face AutoModelForSeq2SeqLM

### Parametreler

- **`pretrained_model_name_or_path`**:
  - Modelin önceden eğitilmiş halinin adını veya yolunu belirtir. Aşağıdaki şekillerde kullanılabilir:
    - Hugging Face model deposundaki bir *model id*'si.
    - Model ağırlıklarını içeren bir klasörün yolu (örn., `./my_model_directory/`).
    - TensorFlow kontrol noktası dosyasına giden bir yol veya URL (örn., `./tf_model/model.ckpt.index`), bu durumda `from_tf` True olarak ayarlanmalıdır ve `config` argümanı verilmelidir.

- **`model_args`**:
  - Modele özel ek argümanlar, alttaki modelin `__init__()` metoduna aktarılır.

- **`config`**:
  - Modelin kullanılacak yapılandırma dosyası. Aşağıdaki durumlarda otomatik olarak yüklenebilir:
    - Model Hugging Face tarafından sağlanan bir modeldir ve *model id* stringi kullanılarak yüklenir.
    - Model [~PreTrainedModel.save_pretrained] kullanılarak kaydedilmişse ve verilen dizin üzerinden yeniden yüklenir.
    - Model yerel bir klasör üzerinden yüklenir ve klasörde *config.json* dosyası bulunur.

- **`state_dict`**:
  - Kayıtlı ağırlık dosyasından yüklenen bir state dictionary yerine kullanılacak bir state dictionary.

- **`cache_dir`**:
  - İndirilen model yapılandırmasının önbelleğe alınacağı dizin.

- **`from_tf`**:
  - Model ağırlıklarını bir TensorFlow kontrol noktası dosyasından yükler (bu seçenek True yapılmalıdır).

- **`force_download`**:
  - Model ağırlıklarının ve yapılandırma dosyalarının tekrar indirilip indirilmeyeceğini belirler (mevcut önbellek dosyaları üzerine yazılır).

- **`resume_download`**:
  - Artık kullanılmayan ve göz ardı edilen bir argümandır. İndirilenler mümkün olduğunda varsayılan olarak devam eder.

- **`proxies`**:
  - Her bir istek için kullanılacak proxy sunucularının sözlüğü. Örnek: `{'http': 'foo.bar:3128', 'https': 'foo.bar:4012'}`.

- **`output_loading_info`**:
  - Eksik anahtarlar, beklenmeyen anahtarlar ve hata mesajlarını içeren bir sözlük döndürülüp döndürülmeyeceğini belirtir.

- **`local_files_only`**:
  - Yalnızca yerel dosyaların kullanılıp kullanılmayacağını belirtir (örn., indirme işlemi yapılmaz).

- **`revision`**:
  - Kullanılacak model versiyonu. Bir dal adı, etiket adı veya commit id'si olabilir.

- **`trust_remote_code`**:
  - Hub'da tanımlanan özel modellerin yerel makinede çalıştırılmasına izin verilip verilmeyeceğini belirtir. Bu seçenek yalnızca güvenilen depolar için True yapılmalıdır.

- **`code_revision`**:
  - Model dışındaki kodlar için kullanılacak belirli revizyon. Bir dal adı, etiket adı veya commit id'si olabilir.

- **`kwargs`**:
  - Yapılandırma nesnesini güncellemek ve modeli başlatmak için kullanılacak diğer ek argümanlar.


In [9]:
model = AutoModelForSeq2SeqLM.from_pretrained(base_model_name)
model = model.to(device)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [10]:
def preprocess_data(data: dict) -> dict:
    '''
    ## Preprocess Data

    Bu fonksiyon, veri kümesini işlemek için kullanılır. Veri kümesi, modelin eğitimi için uygun hale getirilir.

    Args:
        - data (dict): Veri kümesi.
    
    Returns:
        - inputs (dict): Modelin girdileri.
    '''
    
    instruction_variants = [
        "Summarize the following text:",
        "Provide a brief summary of the following document:",
        "Write a concise summary of the text below:",
        "Can you summarize the main points of the text below?",
        "Summarize the content of the following article:",
        "Provide a short and clear summary for the given text:",
        "What is the summary of this article?",
        "Give a brief summary of the following text:"
    ]
    
    document = [
        f'{random.choice(instruction_variants)} {text}' for text in data['document']
    ]
    summary = data['summary']
    
    inputs = tokenizer(document, truncation=True, padding='max_length', max_length=256, return_tensors='pt')
    outputs = tokenizer(summary, truncation=True, padding='max_length', max_length=128, return_tensors='pt')
    
    inputs['labels'] = outputs['input_ids']
    
    return inputs

In [11]:
train_dataset = train_dataset.map(preprocess_data, batched=True, remove_columns=train_dataset.column_names)
eval_dataset = eval_dataset.map(preprocess_data, batched=True, remove_columns=eval_dataset.column_names)

Map:   0%|          | 0/1632 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

## 🤗 Hugging Face Seq2SeqTrainingArguments Kütüphanesi

### Parametreler

- **`output_dir`**:
  - Model tahminlerinin ve kontrol noktalarının kaydedileceği dizin.

- **`overwrite_output_dir`**:
  - Eğer True ise, `output_dir` içeriği üzerine yazılır.

- **`do_train`**:
  - Eğitim işleminin yapılıp yapılmayacağını belirtir.

- **`do_eval`**:
  - Değerlendirme işleminin yapılıp yapılmayacağını belirtir.

- **`do_predict`**:
  - Tahmin işleminin yapılıp yapılmayacağını belirtir.

- **`eval_strategy`**:
  - Değerlendirme stratejisi: 'no', 'steps' veya 'epoch'.

- **`prediction_loss_only`**:
  - Eğer True ise, sadece kayıp hesaplanır, tahminler döndürülmez.

- **`per_device_train_batch_size`**:
  - Her bir cihaz (GPU/CPU) için eğitim batch boyutu.

- **`per_device_eval_batch_size`**:
  - Her bir cihaz (GPU/CPU) için değerlendirme batch boyutu.

- **`gradient_accumulation_steps`**:
  - Gradient biriktirme adımları. Yüksek değerler bellek kullanımını azaltır ama eğitim süresini uzatır.

- **`eval_accumulation_steps`**:
  - Değerlendirme biriktirme adımları. Yüksek değerler bellek kullanımını azaltır ama değerlendirme süresini uzatır.

- **`eval_delay`**:
  - Eğitim başladıktan sonra ilk değerlendirmenin yapılacağı adım sayısı.

- **`learning_rate`**:
  - Öğrenme oranı. Yüksek değerler daha hızlı öğrenir ama aşırı öğrenmeye (overfitting) yol açabilir.

- **`weight_decay`**:
  - Ağırlıkların küçülme oranı. Yüksek değerler modelin genelleme yeteneğini artırabilir ama öğrenme yavaşlar.

- **`adam_beta1`**:
  - Adam optimizasyon algoritması için beta1 parametresi.

- **`adam_beta2`**:
  - Adam optimizasyon algoritması için beta2 parametresi.

- **`adam_epsilon`**:
  - Adam optimizasyon algoritması için epsilon parametresi.

- **`max_grad_norm`**:
  - Gradientlerin maksimum normu. Büyük değerler modelin stabilitesini artırabilir.

- **`num_train_epochs`**:
  - Eğitim için epoch sayısı. Yüksek değerler daha fazla öğrenme sağlar ama aşırı öğrenmeye yol açabilir.

- **`max_steps`**:
  - Maksimum eğitim adım sayısı. -1 ise, tüm epoch'lar tamamlanır.

- **`lr_scheduler_type`**:
  - Öğrenme oranı zamanlayıcı tipi.

- **`lr_scheduler_kwargs`**:
  - Öğrenme oranı zamanlayıcısı için ek parametreler.

- **`warmup_ratio`**:
  - Öğrenme oranı ısınma oranı. Yüksek değerler başlangıçta daha yavaş öğrenme sağlar.

- **`warmup_steps`**:
  - Öğrenme oranı ısınma adım sayısı. Yüksek değerler başlangıçta daha yavaş öğrenme sağlar.

- **`log_level`**:
  - Log seviyesi. 'passive', 'info', 'warning', 'error' veya 'critical'.

- **`log_level_replica`**:
  - Çoklu GPU eğitiminde log seviyesi.

- **`log_on_each_node`**:
  - Çoklu node eğitiminde her node için loglama yapılır.

- **`logging_dir`**:
  - TensorBoard logları için dizin.

- **`logging_strategy`**:
  - Loglama stratejisi: 'no', 'steps' veya 'epoch'.

- **`logging_first_step`**:
  - Eğer True ise, ilk adımda loglama yapılır.

- **`logging_steps`**:
  - Kaç adımda bir loglama yapılacağı.

- **`logging_nan_inf_filter`**:
  - Eğer True ise, NaN ve sonsuz değerler loglanmaz.

- **`save_strategy`**:
  - Modelin kaydedilme stratejisi: 'no', 'steps' veya 'epoch'.

- **`save_steps`**:
  - Kaç adımda bir modelin kaydedileceği.

- **`save_total_limit`**:
  - Maksimum kaç model kaydedileceği.

- **`save_safetensors`**:
  - Modelin güvenli tensör formatında kaydedilip kaydedilmeyeceği.

- **`save_on_each_node`**:
  - Eğer True ise, her node'da model kaydedilir.

- **`save_only_model`**:
  - Eğer True ise, sadece model kaydedilir, optimizer ve lr scheduler kaydedilmez.

- **`restore_callback_states_from_checkpoint`**:
  - Callback durumlarının checkpoint'ten geri yüklenip yüklenmeyeceği.

- **`no_cuda`**:
  - Eğer True ise, GPU kullanılmaz.

- **`use_cpu`**:
  - Eğer True ise, CPU kullanılır.

- **`use_mps_device`**:
  - Eğer True ise, MacOS Metal Performance Shaders kullanılır.

- **`seed`**:
  - Rastgelelik için seed değeri.

- **`data_seed`**:
  - Veri yükleme için seed değeri.

- **`jit_mode_eval`**:
  - Eğer True ise, PyTorch JIT modu kullanılır.

- **`use_ipex`**:
  - Eğer True ise, Intel Extension for PyTorch kullanılır.

- **`bf16`**:
  - Eğer True ise, bfloat16 kullanılır.

- **`fp16`**:
  - Eğer True ise, float16 kullanılır.

- **`fp16_opt_level`**:
  - float16 optimizasyon seviyesi.

- **`half_precision_backend`**:
  - Yarı hassasiyet arka ucu: 'auto', 'amp' veya 'apex'.

- **`bf16_full_eval`**:
  - Eğer True ise, bfloat16 tam değerlendirme yapılır.

- **`fp16_full_eval`**:
  - Eğer True ise, float16 tam değerlendirme yapılır.

- **`tf32`**:
  - Eğer True ise, TensorFloat-32 kullanılır.

- **`local_rank`**:
  - Çoklu GPU eğitiminde lokal rank.

- **`ddp_backend`**:
  - DDP backend tipi: 'nccl', 'gloo', 'mpi'.

- **`tpu_num_cores`**:
  - TPU çekirdek sayısı.

- **`tpu_metrics_debug`**:
  - TPU metriklerinin debug bilgilerini içerir.

- **`debug`**:
  - Debug seçeneği: 'underflow_overflow', 'tpu_metrics_debug'.

- **`dataloader_drop_last`**:
  - Eğer True ise, dataloader son batch'i düşürür.

- **`eval_steps`**:
  - Değerlendirme adım sayısı.

- **`dataloader_num_workers`**:
  - Dataloader için çalışan sayısı. Yüksek değerler hız artırır ama daha fazla bellek kullanır.

- **`dataloader_prefetch_factor`**:
  - Dataloader için prefetch faktörü. Yüksek değerler hız artırır ama daha fazla bellek kullanır.

- **`past_index`**:
  - Geçmiş index.

- **`run_name`**:
  - Çalışma ismi.

- **`disable_tqdm`**:
  - Eğer True ise, tqdm progress bar devre dışı bırakılır.

- **`remove_unused_columns`**:
  - Eğer True ise, kullanılmayan sütunlar veri setinden kaldırılır.

- **`label_names`**:
  - Label isimleri.

- **`load_best_model_at_end`**:
  - Eğitim sonunda en iyi modelin yüklenip yüklenmeyeceği.

- **`metric_for_best_model`**:
  - En iyi modelin seçilmesi için kullanılacak metrik.

- **`greater_is_better`**:
  - Eğer True ise, metriklerde yüksek değerler daha iyi olarak kabul edilir.

- **`ignore_data_skip`**:
  - Eğer True ise, veri atlamaları göz ardı edilir.

- **`fsdp`**:
  - Fully Sharded Data Parallel ayarları.

- **`fsdp_min_num_params`**:
  - Fully Sharded Data Parallel için minimum parametre sayısı.

- **`fsdp_config`**:
  - Fully Sharded Data Parallel için ek ayarlar.

- **`fsdp_transformer_layer_cls_to_wrap`**:
  - Fully Sharded Data Parallel için transformer layer sınıfı.

- **`accelerator_config`**:
  - Accelerator için ek ayarlar.

- **`deepspeed`**:
  - DeepSpeed yapılandırması.

- **`label_smoothing_factor`**:
  - Label smoothing faktörü.

- **`optim`**:
  - Optimizasyon algoritması.

- **`optim_args`**:
  - Optimizasyon algoritması için ek argümanlar.

- **`adafactor`**:
  - Eğer True ise, Adafactor optimizasyon algoritması kullanılır.

- **`group_by_length`**:
  - Eğer True ise, input uzunluğuna göre gruplanır.

- **`length_column_name`**:
  - Uzunluk sütunu ismi.

- **`report_to`**:
  - Raporlama platformları (örneğin 'wandb').

- **`ddp_find_unused_parameters`**:
  - DDP'de kullanılmayan parametrelerin bulunması.

- **`ddp_bucket_cap_mb`**:
  - DDP bucket kapasitesi.

- **`ddp_broadcast_buffers`**:
  - DDP broadcast buffer'ları.

- **`dataloader_pin_memory`**:
  - Dataloader için bellek pinleme.

- **`dataloader_persistent_workers`**:
  - Dataloader için kalıcı çalışanlar.

- **`skip_memory_metrics`**:
  - Eğer True ise, bellek metrikleri atlanır.

- **`use_legacy_prediction_loop`**:
  - Eğer True ise, eski tahmin döngüsü kullanılır.

- **`push_to_hub`**:
  - Eğer True ise, model Hugging Face Hub'a itilir.

- **`resume_from_checkpoint`**:
  - Checkpoint'ten devam edilir.

- **`hub_model_id`**:
  - Hugging Face Hub model ID.

- **`hub_strategy`**:
  - Hub stratejisi.

- **`hub_token`**:
  - Hugging Face Hub token.

- **`hub_private_repo`**:
  - Eğer True ise, özel repo kullanılır.

- **`hub_always_push`**:
  - Eğer True ise, her zaman Hub'a itilir.

- **`gradient_checkpointing`**:
  - Eğer True ise, gradient checkpointing yapılır.

- **`gradient_checkpointing_kwargs`**:
  - Gradient checkpointing için ek argümanlar.

- **`include_inputs_for_metrics`**:
  - Eğer True ise, değerlendirme metrikleri için inputlar da dahil edilir.

- **`eval_do_concat_batches`**:
  - Eğer True ise, değerlendirme batch'leri birleştirilir.

- **`fp16_backend`**:
  - float16 arka ucu.

- **`evaluation_strategy`**:
  - Değerlendirme stratejisi: 'no', 'steps' veya 'epoch'.

- **`push_to_hub_model_id`**:
  - Hub model ID.

- **`push_to_hub_organization`**:
  - Hub organizasyon ID.

- **`push_to_hub_token`**:
  - Hub token.

- **`mp_parameters`**:
  - Model paralel parametreler.

- **`auto_find_batch_size`**:
  - Eğer True ise, batch boyutu otomatik bulunur.

- **`full_determinism`**:
  - Eğer True ise, deterministik eğitim yapılır.

- **`torchdynamo`**:
  - TorchDynamo kullanımı.

- **`ray_scope`**:
  - Ray scope.

- **`ddp_timeout`**:
  - DDP timeout süresi.

- **`torch_compile`**:
  - Eğer True ise, Torch compile kullanılır.

- **`torch_compile_backend`**:
  - Torch compile backend.

- **`torch_compile_mode`**:
  - Torch compile modu.

- **`dispatch_batches`**:
  - Batch dispatching.

- **`split_batches`**:
  - Batch splitting.

- **`include_tokens_per_second`**:
  - Eğer True ise, saniye başına token sayısı dahil edilir.

- **`include_num_input_tokens_seen`**:
  - Eğer True ise, görülen input token sayısı dahil edilir.

- **`neftune_noise_alpha`**:
  - Neptune noise alpha.

- **`optim_target_modules`**:
  - Optimizasyon hedef modüller.

- **`batch_eval_metrics`**:
  - Eğer True ise, batch değerlendirme metrikleri hesaplanır.

In [12]:
training_args = Seq2SeqTrainingArguments(
    output_dir='./results', 
    num_train_epochs=10, 
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8, 
    report_to='none',
    weight_decay=0.01, 
    hub_model_id='t5-small-instruct-summarizer'
)

## 🤗 Hugging Face Seq2SeqTrainer Kütüphanesi

### Parametreler

- **`model`**:
  - Eğitim ve değerlendirme için kullanılacak model. `PreTrainedModel` veya `torch.nn.Module` olmalıdır. Eğer belirtilmezse, bir `model_init` fonksiyonu verilmelidir.

- **`args`**:
  - Eğitim argümanları, `TrainingArguments` sınıfı ile oluşturulur. Eğer belirtilmezse, `output_dir` parametresi *tmp_trainer* olan temel bir `TrainingArguments` örneği kullanılır.

- **`data_collator`**:
  - Veriyi toplamak ve batch oluşturmak için kullanılan fonksiyon. Eğer belirtilmezse, `tokenizer` yoksa `default_data_collator`, varsa `DataCollatorWithPadding` kullanılır.

- **`train_dataset`**:
  - Eğitim için kullanılacak veri seti. `torch.utils.data.Dataset`, `torch.utils.data.IterableDataset` veya `datasets.Dataset` olabilir.

- **`eval_dataset`**:
  - Değerlendirme için kullanılacak veri seti. `torch.utils.data.Dataset`, `datasets.Dataset` veya bunların sözlük hali olabilir.

- **`tokenizer`**:
  - Metin verilerini modelin anlayabileceği sayısal değerlere dönüştürür. Eğer belirtilirse, girdileri otomatik olarak maksimum uzunluğa kadar pad eder ve modelle birlikte kaydedilir.

- **`model_init`**:
  - Modelin başlatılması için kullanılacak fonksiyon. Her `train` çağrısında yeni bir model örneği başlatmak için kullanılır.

- **`compute_metrics`**:
  - Değerlendirme metriklerini hesaplamak için kullanılacak fonksiyon. `EvalPrediction` alır ve metrik değerlerine sahip bir sözlük döndürür.

- **`callbacks`**:
  - Eğitim sürecini izlemek ve kontrol etmek için kullanılacak callback listesi. Varsayılan callback'lere ek olarak kullanılır.

- **`optimizers`**:
  - Optimizasyon algoritması ve öğrenme oranı zamanlayıcısı. Varsayılan olarak, `AdamW` optimizer ve `get_linear_schedule_with_warmup` zamanlayıcısı kullanılır.

- **`preprocess_logits_for_metrics`**:
  - Metrik hesaplama için logits'i ön işleme fonksiyonu. İki tensor alır, logits ve etiketler, ve işlenen logits'i döndürür. Etiketler (`labels`) eğer veri setinde yoksa `None` olabilir.


In [13]:
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=tokenizer, 
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    args=training_args
)

In [14]:
trainer.train()

Step,Training Loss
500,1.329300
1000,0.685200


TrainOutput(global_step=1020, training_loss=1.0009192448036344, metrics={'train_runtime': 297.3561, 'train_samples_per_second': 54.884, 'train_steps_per_second': 3.43, 'total_flos': 1104389100011520.0, 'train_loss': 1.0009192448036344, 'epoch': 10.0})

In [20]:
trainer.push_to_hub()

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/fawern/t5-small-instruct-summarizer/commit/6bd60e4b6663abe5c935cad370d75dce3f865a1b', commit_message='End of training', commit_description='', oid='6bd60e4b6663abe5c935cad370d75dce3f865a1b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/fawern/t5-small-instruct-summarizer', endpoint='https://huggingface.co', repo_type='model', repo_id='fawern/t5-small-instruct-summarizer'), pr_revision=None, pr_num=None)

In [15]:
trained_model_path = 'trained_model'

model.save_pretrained(trained_model_path)
tokenizer.save_pretrained(trained_model_path)

('trained_model/tokenizer_config.json',
 'trained_model/special_tokens_map.json',
 'trained_model/spiece.model',
 'trained_model/added_tokens.json',
 'trained_model/tokenizer.json')

## 🤗 Hugging Face Pipeline

### Parametreler

- **`task`**:
  - Geri döndürülecek pipeline türünü tanımlar. Şu anda kabul edilen görevler şunlardır:
    - `"audio-classification"`: [`AudioClassificationPipeline`] döndürür.
    - `"automatic-speech-recognition"`: [`AutomaticSpeechRecognitionPipeline`] döndürür.
    - `"depth-estimation"`: [`DepthEstimationPipeline`] döndürür.
    - `"document-question-answering"`: [`DocumentQuestionAnsweringPipeline`] döndürür.
    - `"feature-extraction"`: [`FeatureExtractionPipeline`] döndürür.
    - `"fill-mask"`: [`FillMaskPipeline`] döndürür.
    - `"image-classification"`: [`ImageClassificationPipeline`] döndürür.
    - `"image-feature-extraction"`: [`ImageFeatureExtractionPipeline`] döndürür.
    - `"image-segmentation"`: [`ImageSegmentationPipeline`] döndürür.
    - `"image-to-image"`: [`ImageToImagePipeline`] döndürür.
    - `"image-to-text"`: [`ImageToTextPipeline`] döndürür.
    - `"mask-generation"`: [`MaskGenerationPipeline`] döndürür.
    - `"object-detection"`: [`ObjectDetectionPipeline`] döndürür.
    - `"question-answering"`: [`QuestionAnsweringPipeline`] döndürür.
    - `"summarization"`: [`SummarizationPipeline`] döndürür.
    - `"table-question-answering"`: [`TableQuestionAnsweringPipeline`] döndürür.
    - `"text2text-generation"`: [`Text2TextGenerationPipeline`] döndürür.
    - `"text-classification"` (alias `"sentiment-analysis"`): [`TextClassificationPipeline`] döndürür.
    - `"text-generation"`: [`TextGenerationPipeline`] döndürür.
    - `"text-to-audio"` (alias `"text-to-speech"`): [`TextToAudioPipeline`] döndürür.
    - `"token-classification"` (alias `"ner"`): [`TokenClassificationPipeline`] döndürür.
    - `"translation"`: [`TranslationPipeline`] döndürür.
    - `"video-classification"`: [`VideoClassificationPipeline`] döndürür.
    - `"visual-question-answering"`: [`VisualQuestionAnsweringPipeline`] döndürür.
    - `"zero-shot-classification"`: [`ZeroShotClassificationPipeline`] döndürür.
    - `"zero-shot-image-classification"`: [`ZeroShotImageClassificationPipeline`] döndürür.
    - `"zero-shot-audio-classification"`: [`ZeroShotAudioClassificationPipeline`] döndürür.
    - `"zero-shot-object-detection"`: [`ZeroShotObjectDetectionPipeline`] döndürür.

- **`model`**:
  - Pipeline tarafından kullanılacak model. Bu, model kimliği (string) ya da [`PreTrainedModel`] veya [`TFPreTrainedModel`] gibi eğitilmiş bir model olabilir. Eğer belirtilmezse, varsayılan model yüklenir.

- **`config`**:
  - Pipeline'ı başlatırken kullanılacak model yapılandırması. Model kimliği veya [`PretrainedConfig`] olabilir. Eğer belirtilmezse, görev için varsayılan yapılandırma kullanılır.

- **`tokenizer`**:
  - Pipeline tarafından kullanılacak tokenizer. Model kimliği veya [`PreTrainedTokenizer`] olabilir. Eğer belirtilmezse, model veya görev için varsayılan tokenizer yüklenir.

- **`feature_extractor`**:
  - Model için veriyi işlemek için kullanılacak feature extractor. Ses veya görüntü gibi NLP dışı modellerde kullanılır. Eğer belirtilmezse, görev için varsayılan extractor yüklenir.

- **`framework`**:
  - Kullanılacak framework, `"pt"` (PyTorch) veya `"tf"` (TensorFlow) olabilir. Eğer belirtilmezse, kurulu olan framework tercih edilir.

- **`revision`**:
  - Bir görev adı veya model kimliği verildiğinde, kullanılacak model sürümünü belirtir (dal adı, etiket adı, commit id'si olabilir).

- **`use_fast`**:
  - Eğer mümkünse, [`PreTrainedTokenizerFast`] kullanılıp kullanılmayacağını belirtir. Varsayılan `True`.

- **`token`**:
  - HTTP yetkilendirme için kullanılacak token. Eğer `True` ise, `huggingface-cli login` ile alınan token kullanılır.

- **`device`**:
  - Pipeline'ın hangi cihazda çalışacağını belirtir. Örneğin `"cpu"`, `"cuda:0"`, `"mps"`, veya GPU sıralaması gibi (örn., `1`).

- **`device_map`**:
  - Model'in belirli parçasını hangi cihazda çalıştırılacağını belirler. `accelerate` kütüphanesi varsa, `device_map="auto"` otomatik olarak optimize edilmiş bir yapı çıkarır.

- **`torch_dtype`**:
  - Model'in hangi tür hassasiyetle çalışacağını belirtir (örn. `torch.float16`, `torch.bfloat16`, `"auto"`).

- **`trust_remote_code`**:
  - Hub'daki özel modellerin yerel makinede çalışmasına izin verilip verilmeyeceğini belirtir. Sadece güvenilir kodlar için kullanılmalıdır.

- **`model_kwargs`**:
  - Model'in `from_pretrained(..., **model_kwargs)` fonksiyonuna geçirilecek ek argümanları içeren sözlük.

- **`kwargs`**:
  - Belirli pipeline'ın `init` fonksiyonuna geçirilecek diğer ek argümanlar.



In [16]:
summarizer = pipeline('summarization', model=trained_model_path, tokenizer=trained_model_path, device=device)

In [17]:
def get_summarized_text(sample_index: int = 0, sample_text: str = None) -> None:
    '''
    ## Get Summarized Text

    Bu fonksiyon, belirli bir örneğin özetlenmiş metnini döndürmek için kullanılır.

    Args:
        - sample_index (int): Örnek dizini.
        - sample_text (str): Örnek metin.
    
    Returns:
        - None
    '''
    if sample_text is None:
        sample = train_dataset[sample_index]

        full_text = tokenizer.decode(sample['input_ids'], skip_special_tokens=True)

        correct_summary = tokenizer.decode(sample['labels'], skip_special_tokens=True)

        generated_summary = summarizer(full_text)[0]['summary_text']

        print(f'Full Document Text:\n{full_text}')
        print('\n' + '-' * 50 + '\n')
        print(f"Correct Target Summarization:\n{correct_summary}")
        print('\n' + '-' * 50 + '\n')
        print(f"Generated Summarization:\n{generated_summary}")
    else:        
        generated_summary = summarizer(sample_text)[0]['summary_text']

        print(f'Full Document Text:\n{sample_text}')
        print('\n' + '-' * 50 + '\n')
        print(f"Generated Summarization:\n{generated_summary}")

In [18]:
get_summarized_text(sample_index=0)

Full Document Text:
Give a brief summary of the following text: Corey Price, Alesha O'Connor and Rhodri Miller, all 17, and Margaret Challis, 66, died in a two-car collision on the A470 in the Brecon Beacons in March. Cardiff City Football Club is hosting the match on Sunday, with teams made up of members of the community. Corey was part of the club's academy for eight years from the age of nine. Entry to the match at Cardiff City Stadium is free but there will be bucket collections on the day. The money will be used to pay for a Corey Price memorial bench at Ysgol Gyfun Bro Morgannwg, in Barry, Vale of Glamorgan, where he was a pupil. A second bench is also planned at the Cardiff City Academy training ground and the club hopes the match will become an annual event. Corey and his two friends, all from Barry, were travelling together when the collision happened near Storey Arms, Brecon on 6 March. Mrs Challis, 66, from Merthyr Tydfil, was a passenger in the second car involved. Seven te

In [19]:
text = """
The quick brown fox jumps over the lazy dog. This sentence is often used to illustrate the use of every letter in the English alphabet. It serves as a great example for typography, fonts, and teaching handwriting. 
Despite its simple structure, the sentence has become widely known in English-speaking countries and is often used as a typing exercise or to test computer keyboards. 
Additionally, it showcases the interplay between common animals, which makes it easy to remember and visually appealing."""

get_summarized_text(sample_text=text)

Your max_length is set to 200, but your input_length is only 106. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=53)


Full Document Text:

The quick brown fox jumps over the lazy dog. This sentence is often used to illustrate the use of every letter in the English alphabet. It serves as a great example for typography, fonts, and teaching handwriting. 
Despite its simple structure, the sentence has become widely known in English-speaking countries and is often used as a typing exercise or to test computer keyboards. 
Additionally, it showcases the interplay between common animals, which makes it easy to remember and visually appealing.

--------------------------------------------------

Generated Summarization:
English-speaking countries have used this sentence to illustrate the use of every letter in the English alphabet . it is often used as a typing exercise or to test computer keyboards, which makes it visually appealing.
